# S1 J5 ? Delta Lake (Write + Read)

This notebook writes a Delta table and reads it back.

If you are running locally and Delta is missing, install it first:
- `pip install delta-spark`


In [1]:
pip install delta-spark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [delta-spark]
Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# spark.stop()


builder = (
    SparkSession.builder
    .appName("delta-lake-demo")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()



In [12]:
from pyspark.sql import functions as F

data_path = "../../data/example.csv"
delta_path = "../../data/delta/users"

raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(data_path)
)

silver = (
    raw
    .withColumn("signup_date", F.to_date("signup_date"))
    .withColumn("spend", F.col("spend").cast("double"))
)


In [13]:
# Write Delta table
(
    silver
    .write
    .format("delta")
    .mode("overwrite")
    .save(delta_path)
)


Py4JJavaError: An error occurred while calling o142.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:764)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:686)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:745)
	at org.apache.spark.sql.classic.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:596)
	at org.apache.spark.sql.classic.DataFrameWriter.saveCommand(DataFrameWriter.scala:141)
	at org.apache.spark.sql.classic.DataFrameWriter.save(DataFrameWriter.scala:115)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMet

In [ ]:
# Read Delta table
delta_df = spark.read.format("delta").load(delta_path)
delta_df.show(truncate=False)


In [ ]:
# Register as a table for SQL queries (optional)
spark.sql(f"DROP TABLE IF EXISTS users_delta")
spark.sql(f"CREATE TABLE users_delta USING DELTA LOCATION '{delta_path}'")

spark.sql("SELECT plan, COUNT(*) AS users, ROUND(SUM(spend), 2) AS total_spend FROM users_delta GROUP BY plan").show()
